In [1]:
import pandas as pd
import numpy as np

In [269]:
features = pd.read_csv('/Users/kuptservol/.datasets/dota-2-win-probability-prediction/features.csv', index_col='match_id')

features.head(3).T

match_id,0,1,2
start_time,1.430199e+09,1.430220e+09,1.430227e+09
lobby_type,7.000000e+00,0.000000e+00,7.000000e+00
r1_hero,1.100000e+01,4.200000e+01,3.300000e+01
r1_level,5.000000e+00,4.000000e+00,4.000000e+00
r1_xp,2.098000e+03,1.188000e+03,1.319000e+03
...,...,...,...
radiant_win,1.000000e+00,1.000000e+00,0.000000e+00
tower_status_radiant,1.796000e+03,1.974000e+03,0.000000e+00
tower_status_dire,0.000000e+00,0.000000e+00,1.830000e+03
barracks_status_radiant,5.100000e+01,6.300000e+01,0.000000e+00


In [270]:
features.shape

(97230, 108)

In [271]:
features.describe(include="all")

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
count,9.723000e+04,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,...,97230.000000,97230.000000,97230.000000,95404.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000
mean,1.444232e+09,2.630999,51.517104,3.442672,1233.405801,1147.899702,11.231996,0.357009,0.362285,8.271315,...,3.349553,2.448339,0.689119,-6.901922,2332.247886,0.518503,1309.227790,1286.310820,40.599095,41.337036
std,5.515393e+06,2.835761,32.564211,1.111741,566.588895,464.111662,9.041620,0.663889,0.626704,2.497575,...,1.155609,0.813459,0.710122,40.701397,715.806850,0.499660,853.921365,851.009148,27.871645,27.064873
min,1.430199e+09,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-84.000000,900.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.440815e+09,1.000000,22.000000,3.000000,767.000000,746.000000,2.000000,0.000000,0.000000,7.000000,...,3.000000,2.000000,0.000000,-31.000000,1818.000000,0.000000,36.000000,256.000000,3.000000,3.000000
50%,1.446338e+09,1.000000,50.000000,3.000000,1175.000000,1113.000000,11.000000,0.000000,0.000000,8.000000,...,3.000000,2.000000,1.000000,-16.000000,2268.000000,1.000000,1824.000000,1798.000000,63.000000,60.000000
75%,1.448829e+09,7.000000,75.000000,4.000000,1704.000000,1479.000000,19.000000,1.000000,1.000000,10.000000,...,4.000000,3.000000,1.000000,8.000000,2778.000000,1.000000,1974.000000,1974.000000,63.000000,63.000000
max,1.450313e+09,7.000000,112.000000,6.000000,3319.000000,4332.000000,47.000000,8.000000,5.000000,34.000000,...,9.000000,9.000000,13.000000,300.000000,8452.000000,1.000000,2047.000000,2047.000000,63.000000,63.000000


In [272]:
# Считаем таблицу с признаками из файла features.csv. 
# Удалим признаки, связанные с итогами матча
X = features.drop(
    ['duration','radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_dire', 'barracks_status_radiant'], axis=1)

In [273]:
# Колонки с пустыми значениями
null_colls = X.columns[X.isnull().sum()>0]; null_colls

Index(['first_blood_time', 'first_blood_team', 'first_blood_player1',
       'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time',
       'radiant_flying_courier_time', 'radiant_first_ward_time',
       'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time',
       'dire_first_ward_time'],
      dtype='object')

Большая часть - это просто события которые не успели произойти в первые 5 минут - логичнее заменять их не на средние значение - а оставить факт пустого поля как фичу. Для деревьев нельзя оставлять nan - заменяем на значение которое отличается от среднего - просто ставим значение за пределами 5 минут.

In [31]:
X[null_colls].describe()

,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_first_ward_time
count,77677.000000,77677.000000,77677.000000,53243.000000,81539.000000,96538.000000,69751.000000,95394.000000,81087.000000,96554.000000,71132.000000,95404.000000
mean,97.688029,0.521815,4.593483,4.383224,126.800255,-80.059510,215.987126,-6.875747,127.215028,-80.191893,214.870536,-6.901922
std,82.625879,0.499527,2.886984,2.870915,61.427907,15.814209,34.604997,39.508650,62.442018,15.261950,34.137158,40.701397
min,-78.000000,0.000000,0.000000,0.000000,-37.000000,-90.000000,180.000000,-236.000000,-45.000000,-90.000000,180.000000,-84.000000
25%,15.000000,0.000000,2.000000,2.000000,84.000000,-86.000000,186.000000,-31.000000,83.000000,-86.000000,185.000000,-31.000000
50%,93.000000,1.000000,5.000000,4.000000,131.000000,-83.000000,204.000000,-15.000000,131.000000,-84.000000,203.000000,-16.000000
75%,155.000000,1.000000,7.000000,7.000000,164.000000,-79.000000,241.000000,9.000000,165.000000,-79.000000,238.000000,8.000000
max,300.000000,1.000000,9.000000,9.000000,300.000000,300.000000,300.000000,300.000000,300.000000,296.000000,300.000000,300.000000


In [34]:
X.fillna(value=1_000_000, inplace=True)

In [69]:
# Целевая переменная
y = features['radiant_win']

Оценивать алгоритмы будем с помощью кросс-валидации с шаффлом - матчи не зависят друг от друга по времени

In [38]:
from sklearn.model_selection import KFold

In [57]:
cv = KFold(shuffle=True, random_state=42)

### Градиентный бустинг

In [40]:
from sklearn.ensemble import GradientBoostingClassifier

In [43]:
from sklearn.model_selection import cross_val_score

In [63]:
import time
import datetime

In [68]:
for num_trees in [1, 5, 10, 20, 30, 40]:
    start_time = datetime.datetime.now()
    cls = GradientBoostingClassifier(n_estimators=num_trees)
    cross_val = cross_val_score(cls, X, y, cv=cv, scoring='roc_auc')
    print("Num trees: ", num_trees, "ROC AUC avg: ", np.average(cross_val), 'Time elapsed:', datetime.datetime.now() - start_time)

Num trees:  1 ROC AUC avg:  0.5921867014628291 Time elapsed: 0:00:04.693902
Num trees:  5 ROC AUC avg:  0.6369304208832456 Time elapsed: 0:00:20.084035
Num trees:  10 ROC AUC avg:  0.6660340264507757 Time elapsed: 0:00:40.067739
Num trees:  20 ROC AUC avg:  0.6834568680262606 Time elapsed: 0:01:21.195295
Num trees:  30 ROC AUC avg:  0.6899489423233114 Time elapsed: 0:02:03.643132
Num trees:  40 ROC AUC avg:  0.6943471950732155 Time elapsed: 0:02:45.497876


Обучение с градиентным бустингом на 30 деревьях заняло 2 минуты - ROC-AUC 0.68. С ростом кол-ва деревьев качество растет - но растет и время обучения.

Для ускорения можно попробовать уменьшить глубину деревьев или сократить размер выборки

In [ ]:
Для ускорения можно попробовать уменьшить глубину деревьев или сократить размер выборки

In [73]:
num_trees = 40
start_time = datetime.datetime.now()
cls = GradientBoostingClassifier(n_estimators=num_trees, max_depth=2)
cross_val = cross_val_score(cls, X, y, cv=cv, scoring='roc_auc')
print("Num trees: ", num_trees, "ROC AUC avg: ", np.average(cross_val), 'Time elapsed:', datetime.datetime.now() - start_time)

Num trees:  40 ROC AUC avg:  0.6893416543397144 Time elapsed: 0:01:39.659411


Получилось качество как при 30 деревьях - но при этом почти в полтора раз быстрее

### Логистическая регресиия

Регресиия плохо работает с признаками разного размаха - поэтому делаем предобработку данных - делаем нормализацию признаков.

In [90]:
from sklearn.preprocessing import StandardScaler

In [91]:
X_scaled = StandardScaler().fit_transform(X)

In [274]:
X_scaled_df = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

In [94]:
from sklearn.linear_model import LogisticRegression

In [95]:
for reg in [0.001, 0.01, 0.1, 1.0, 10, 100]:
    start_time = datetime.datetime.now()
    cls = LogisticRegression(C=reg)
    cross_val = cross_val_score(cls, X_scaled_df, y, cv=cv, scoring='roc_auc')
    print("C: ", reg, "ROC AUC avg: ", np.average(cross_val), 'Time elapsed:', datetime.datetime.now() - start_time)

C:  0.001 ROC AUC avg:  0.7156041952303434 Time elapsed: 0:00:01.708015
C:  0.01 ROC AUC avg:  0.7158122095189238 Time elapsed: 0:00:02.190793
C:  0.1 ROC AUC avg:  0.7157883808878556 Time elapsed: 0:00:02.158844
C:  1.0 ROC AUC avg:  0.7157863704385801 Time elapsed: 0:00:02.299518
C:  10 ROC AUC avg:  0.7157865483247136 Time elapsed: 0:00:02.203682
C:  100 ROC AUC avg:  0.715786255938961 Time elapsed: 0:00:02.391824


Наилучшее качество получилось при C=0.01 roc_auc = 0.71581 - что выше чем у градиентного бустинга - и в несколько раз быстрее - 2 минуты у бустинга против 2 секунд у регрессии.
Разница в качестве может быть объяснена большим кол-вом признаков - часть из которых не влияет на реузультат - вероятно градиентный бустинг плохо это улавливает.

Мы использовали категориальные признаки как числовые - что не очень хорошо - так как линейный алгоритм начинает учитывать расстояние между значениями признака. Попробуем убрать категориальные признаки совсем и проверить качество.

In [97]:
X_wot_cat = X_scaled_df.drop(['lobby_type', 'r1_hero', 'r2_hero', 
                              'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                              'd3_hero', 'd4_hero', 'd5_hero'], axis=1)

In [98]:
for reg in [0.001, 0.01, 0.1, 1.0, 10, 100]:
    start_time = datetime.datetime.now()
    cls = LogisticRegression(C=reg)
    cross_val = cross_val_score(cls, X_wot_cat, y, cv=cv, scoring='roc_auc')
    print("C: ", reg, "ROC AUC avg: ", np.average(cross_val), 'Time elapsed:', datetime.datetime.now() - start_time)

C:  0.001 ROC AUC avg:  0.7156309721332736 Time elapsed: 0:00:01.578485
C:  0.01 ROC AUC avg:  0.7158340794594586 Time elapsed: 0:00:02.039179
C:  0.1 ROC AUC avg:  0.7158133379331005 Time elapsed: 0:00:02.138344
C:  1.0 ROC AUC avg:  0.7158104102155597 Time elapsed: 0:00:02.249685
C:  10 ROC AUC avg:  0.7158095438217161 Time elapsed: 0:00:02.192774
C:  100 ROC AUC avg:  0.7158094823629536 Time elapsed: 0:00:02.177348


Как мы видим - качество даже повысилось до 0.71583 - вероятно это можно объяснить тем, что алгоритм некорректно использует числовое расстояние между значениями категориального признака - которое не отражает зависимость реального мира.

In [147]:
heroes = pd.read_csv('/Users/kuptservol/.datasets/dota-2-win-probability-prediction/dictionaries/heroes.csv')

In [150]:
# Кол-во уникальных значений идентификаторов героев
N = len(heroes); N

112

Начнем использовать категориальные признаки в виде "one hot encoding"

In [151]:
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X['r%d_hero' % (p+1)][match_id]-1] = 1
        X_pick[i, X['d%d_hero' % (p+1)][match_id]-1] = -1

In [164]:
X_pick.shape

(97230, 112)

In [185]:
X_with_cat_scaled = X.drop(['lobby_type', 'r1_hero', 'r2_hero', 
                              'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                              'd3_hero', 'd4_hero', 'd5_hero'], axis=1)

In [186]:
X_pick.shape

(97230, 112)

In [187]:
X_with_cat_scaled.values.shape

(97230, 91)

In [190]:
X_with_cat_scaled = pd.DataFrame(np.hstack([X_wot_cat.values,X_pick]))

In [191]:
X_with_cat_scaled.shape

(97230, 203)

In [193]:
for reg in [0.001, 0.01, 0.1, 1.0, 10, 100]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=reg, max_iter=1000)
    cross_val = cross_val_score(clf, X_with_cat_scaled, y, cv=cv, scoring='roc_auc')
    print("C: ", reg, "ROC AUC avg: ", np.average(cross_val), 'Time elapsed:', datetime.datetime.now() - start_time)

C:  0.001 ROC AUC avg:  0.7459758982415087 Time elapsed: 0:00:02.742952
C:  0.01 ROC AUC avg:  0.7514023849846059 Time elapsed: 0:00:04.841362
C:  0.1 ROC AUC avg:  0.7516026936887619 Time elapsed: 0:00:09.035607
C:  1.0 ROC AUC avg:  0.7515853297183722 Time elapsed: 0:00:09.355071
C:  10 ROC AUC avg:  0.7515832954098551 Time elapsed: 0:00:09.240704
C:  100 ROC AUC avg:  0.7515820602648153 Time elapsed: 0:00:09.225738


ROC-AUC увеличился до 0.75160

### Делаем предсказание для финальной модели

In [246]:
# фиксируем финальную модель
clf = LogisticRegression(C=0.1, max_iter=1000)
clf.fit(X_with_cat_scaled, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Делаем препроцессинг аналогичный train-сету

In [218]:
X_test = pd.read_csv('/Users/kuptservol/.datasets/dota-2-win-probability-prediction/test_features.csv', index_col='match_id')

In [219]:
X_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [230]:
X_test.fillna(value=1_000_000, inplace=True)

In [231]:
X_test_scaled = (pd.DataFrame(StandardScaler().fit_transform(X_test), 
                             index=X_test.index, columns=X_test.columns))

In [233]:
X_test_scaled_wot_cat = X_test_scaled.drop(['lobby_type', 'r1_hero', 'r2_hero', 
                              'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                              'd3_hero', 'd4_hero', 'd5_hero'], axis=1)

In [234]:
X_test_scaled_wot_cat.shape

(17177, 91)

In [235]:
X_pick_test = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick_test[i, X_test['r%d_hero' % (p+1)][match_id]-1] = 1
        X_pick_test[i, X_test['d%d_hero' % (p+1)][match_id]-1] = -1

In [236]:
X_pick_test.shape

(17177, 112)

In [243]:
X_test_with_cat_scaled = pd.DataFrame(np.hstack([X_test_scaled_wot_cat.values,X_pick_test]))

In [247]:
test_predicts = clf.predict_proba(X_test_with_cat_scaled)[:, 1]

In [249]:
test_predicts.shape

(17177,)

In [ ]:
test_results = pd.DataFrame([''])

### Заливаем решение на kaggle

In [258]:
subm = pd.read_csv('/Users/kuptservol/.datasets/dota-2-win-probability-prediction/sample_submission.csv')

In [260]:
subm.radiant_win = test_predicts

In [261]:
subm

,match_id,radiant_win
0,6,0.811416
1,7,0.749962
2,10,0.188183
3,13,0.870110
4,16,0.247553
...,...,...
17172,114369,0.716152
17173,114377,0.612581
17174,114378,0.236998
17175,114393,0.665357


In [262]:
subm.to_csv('/tmp/dota-subm.csv', index=None)

In [268]:
! kaggle competitions submit -c dota-2-win-probability-prediction -f /tmp/dota-subm.csv -m 'first'

100%|████████████████████████████████████████| 423k/423k [00:06<00:00, 63.8kB/s]
Successfully submitted to Dota 2: Win Probability Prediction

In [275]:
subm['radiant_win'].describe()

count    17177.000000
mean         0.518297
std          0.221170
min          0.008333
25%          0.346776
50%          0.524121
75%          0.692575
max          0.996451
Name: radiant_win, dtype: float64